In [1]:
import pandas as pd
from sklearn import preprocessing

In [2]:
df = pd.read_csv('data\credit_card_transactions.csv')

<>:1: SyntaxWarning: invalid escape sequence '\c'
<>:1: SyntaxWarning: invalid escape sequence '\c'
/var/folders/4p/rmrxtvt936v30345zcy84nw40000gn/T/ipykernel_23634/3307746668.py:1: SyntaxWarning: invalid escape sequence '\c'
  df = pd.read_csv('data\credit_card_transactions.csv')
/var/folders/4p/rmrxtvt936v30345zcy84nw40000gn/T/ipykernel_23634/3307746668.py:1: SyntaxWarning: invalid escape sequence '\c'
  df = pd.read_csv('data\credit_card_transactions.csv')


FileNotFoundError: [Errno 2] No such file or directory: 'data\\credit_card_transactions.csv'

In [ ]:
df_dropped = df.copy()

In [ ]:
df_dropped = df_dropped.drop(columns=['Unnamed: 0','cc_num','first', 'last', 'gender', 'street', 'lat', 'long', 'dob','trans_num','merch_lat', 'merch_long', 'unix_time'])

In [ ]:
df_dropped['merch_zipcode'] = df_dropped['merch_zipcode'].fillna(df_dropped['zip'])

In [ ]:
df_copy = df_dropped.copy()
df_copy = df_copy[df_copy['amt'] < 10000]

In [ ]:
le = preprocessing.LabelEncoder()
#df_copy['category'] = le.fit_transform(df_copy['category'])
dummies = pd.get_dummies(df_copy['category'], drop_first=True, dtype=int)
df_copy = pd.concat([df_copy, dummies], axis=1)

In [ ]:
df_copy = df_copy.drop(columns=['category'])
df_copy.head()

,trans_date_trans_time,merchant,amt,city,state,zip,city_pop,job,is_fraud,merch_zipcode,...,grocery_pos,health_fitness,home,kids_pets,misc_net,misc_pos,personal_care,shopping_net,shopping_pos,travel
0,2019-01-01 00:00:18,"fraud_Rippin, Kub and Mann",4.97,Moravian Falls,NC,28654,3495,"Psychologist, counselling",0,28705.0,...,0,0,0,0,1,0,0,0,0,0
1,2019-01-01 00:00:44,"fraud_Heller, Gutmann and Zieme",107.23,Orient,WA,99160,149,Special educational needs teacher,0,99160.0,...,1,0,0,0,0,0,0,0,0,0
2,2019-01-01 00:00:51,fraud_Lind-Buckridge,220.11,Malad City,ID,83252,4154,Nature conservation officer,0,83236.0,...,0,0,0,0,0,0,0,0,0,0
3,2019-01-01 00:01:16,"fraud_Kutch, Hermiston and Farrell",45.00,Boulder,MT,59632,1939,Patent attorney,0,59632.0,...,0,0,0,0,0,0,0,0,0,0
4,2019-01-01 00:03:06,fraud_Keeling-Crist,41.96,Doe Hill,VA,24433,99,Dance movement psychotherapist,0,22844.0,...,0,0,0,0,0,1,0,0,0,0


In [ ]:
df_copy['job'].value_counts()

job
Film/video editor             9779
Exhibition designer           9199
Naval architect               8683
Surveyor, land/geomatics      8680
Materials engineer            8270
                              ... 
Information officer              8
Veterinary surgeon               8
Ship broker                      7
Contracting civil engineer       7
Warehouse manager                7
Name: count, Length: 494, dtype: int64